<a href="https://colab.research.google.com/github/scastillof/Colab/blob/main/Entrega_02_Scrub_Sebasti%C3%A1n_Castillo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Proyecto Final - Reconocimiento de Actividad Humana (HAR)


In [30]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder

###Entrega 01 - OBTAIN

Incluir todo el código de la Etapa 01 - OBTAIN creada en la semana anterior

In [34]:
data = pd.read_csv('/content/WISDM_ar_v1.1_raw.txt', on_bad_lines='skip', header=None) # se crea el dataframe a partir del documento txt proporcionado y subido
# se utiliza el comando on_bad_lines con la instrucción 'skip' la cual salta las lineas con error para no producir un error de lectura.

In [35]:
data.head() # Se utiliza el comando Head para visualizar su estructura 

,0,1,2,3,4,5
0,33,Jogging,49105962326000,-0.694638,12.680544,0.50395286;
1,33,Jogging,49106062271000,5.012288,11.264028,0.95342433;
2,33,Jogging,49106112167000,4.903325,10.882658,-0.08172209;
3,33,Jogging,49106222305000,-0.612916,18.496431,3.0237172;
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164;


In [36]:
data[5] = data[5].str.replace(";", "").astype(float) # considerando que los valores de la columna en el indice 5 incluyen ';' , no podrán ser transformados en valores flotantes.
# Por lo tanto se extrae el ';' con el comando replace().

###Entrega 02 - SCRUB

Eliminar filas con datos faltantes

*Uno de los pasos iniciales en la limpieza de datos consiste en eliminar aquellas filas que contengan datos faltantes. Estas líneas generalmente no aportan información y pueden complicar la etapa de entrenamiento y validación de muchos modelos predictivos.*


In [37]:
data.isnull().values.any() # Se utiliza el comando .isnull para verificar si existen valores nulos

True

In [38]:
data=data.dropna(axis=0) # eliminamos los datos nulos 

Agregar al DataFrame los nombres de las columnas 

*En ocasiones al crear los data frames el sistema asigna nombres genéricos y correlativos a las columnas. Es recomendable renombrar las columnas por nombres significativos para favorecer el análisis posterior de los datos.*

In [39]:
data.columns=['Usuario','Actividad','Tiempo','x','y','z'] # se agregan los nombres a las columnas respectivas

Visualizar las 5 primeras filas del DataFrame (.head())

*Siempre es buena medida visualizar algunas líneas del dataframe para tener una idea concreta del contenido.*


In [40]:
data.head()

,Usuario,Actividad,Tiempo,x,y,z
0,33,Jogging,49105962326000,-0.694638,12.680544,0.503953
1,33,Jogging,49106062271000,5.012288,11.264028,0.953424
2,33,Jogging,49106112167000,4.903325,10.882658,-0.081722
3,33,Jogging,49106222305000,-0.612916,18.496431,3.023717
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164


Buscar datos nulos y eliminarlos si es que existen

In [41]:
data.isnull().sum()

Usuario      0
Actividad    0
Tiempo       0
x            0
y            0
z            0
dtype: int64

Visualizar los tipo de datos a los que pertenece cada columna (.info())

*Es necesario tener un detalle de cuantas variables están considerando y el tipo de datos que pertenecen para verificar si es posible utilizarla directamente para el modelo predictivo o si es necesario realizar una transformación del tipo de dato.*


In [10]:
data.info() #se verifica el detalle de las variables utilizadas

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1086469 entries, 0 to 1086469
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   Usuario    1086469 non-null  int64  
 1   Actividad  1086469 non-null  object 
 2   Tiempo     1086469 non-null  int64  
 3   x          1086469 non-null  float64
 4   y          1086469 non-null  float64
 5   z          1086469 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 58.0+ MB


Cambiar el tipo de datos de las variables predictoras a valores reales, para realizar la clasificación 

*Muchas de las variables contenidas en el DataFrame originalmente se encuentran en el tipo de dato 'object'. En ese formato las variables no pueden ser interpretadas como vectores. Recordemos que la mayoría de los modelos de Machine Learning estudiados operan en el espacio vectorial. Por esta razón las variables que formarán parte del modelo predictivo deben ser transformadas a un formato numérico como 'Float' o 'integer'.*

Para este proyecto en particular se pide transformar  al tipo de dato Float, las variables del acelerómetro en el eje $x$, $y$ y $z$



In [11]:
data.dtypes # Al extraer el ';' en los pasos anteriores se cambio la columna 'Z' a Float, por lo que podemos verificiar
# que X , Y y Z son del tipo flotante.

Usuario        int64
Actividad     object
Tiempo         int64
x            float64
y            float64
z            float64
dtype: object

Imprimir los tipos de datos de las variables del DataFrame

*Se recomienda volver a ejecutar el comando info para verificar que efectivamente las variables cambiaron su tipo de dato y que ahora se pueden procesar con los modelos predictivos.*


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1086469 entries, 0 to 1086469
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   Usuario    1086469 non-null  int64  
 1   Actividad  1086469 non-null  object 
 2   Tiempo     1086469 non-null  int64  
 3   x          1086469 non-null  float64
 4   y          1086469 non-null  float64
 5   z          1086469 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 58.0+ MB


Eliminar las columnas que no se utilizaran para la clasificación

*En este caso deberíamos conservar la actividad respectiva y las variables asociadas al acelerómetro*


In [13]:
data=data.drop(['Usuario', 'Tiempo'], axis=1) # Se eliminan las columnas 'Usuario' y 'Tiempo'

Utilizar el comando *value_counts* para obtener un detalle de cuantas instancias se tienen por actividad. El objetivo consiste en identificar la presencia del balance o desbalance en las categorías a modelar


In [16]:
data.Actividad.value_counts() #Se puede apreciar el desbalance en las categorias que se van a modelar

Walking       418393
Jogging       336448
Upstairs      122869
Downstairs    100426
Sitting        59939
Standing       48394
Name: Actividad, dtype: int64

Este proyecto considera datos con un desbalance de clases. Es necesario balancearlos para que los modelos operen de manera adecuada. Existen muchas maneras de realizar un balance, pero en este proyecto utilizaremos una estrategia sencilla e intuitiva. Simplemente debe identificar la actividad que contenga el número menor de instancias. Se debe realizar un muestreo para las otras actividades utilizando ese valor. De esta manera todas las clases tendrán la misma cantidad de instancias. 



In [20]:
Walking = data[data['Actividad']=='Walking'].head(48394).copy() 
Jogging = data[data['Actividad']=='Jogging'].head(48394).copy()
Upstairs = data[data['Actividad']=='Upstairs'].head(48394).copy()
Downstairs = data[data['Actividad']=='Downstairs'].head(48394).copy()
Sitting = data[data['Actividad']=='Sitting'].head(48394).copy()
Standing = data[data['Actividad']=='Standing'].copy()
# conociendo el menor valor de variables que sería 'Standing'
# Se utiliza este valor para generar el nuevo data balanceado y se extraen los primeros valores en orden.

Establecer el número de instancias y dimensiones del data frame resultante *(shape)*


In [21]:
data_balanceado = pd.DataFrame()
data_balanceado = data_balanceado.append([Walking, Jogging, Upstairs, Downstairs, Sitting, Standing])
data_balanceado.shape
# se crea el nuevo DataFrame y se agregan las instancias extraidas en el paso anterior
# se utiliza el comando shape para mostrar las instancias y la dimensión

(290364, 4)

Volver a imprimir la cantidad de instancias por categoría para verificar que todo esté en orden. *(value_counts)*

In [22]:
data_balanceado.Actividad.value_counts() # se verifica la cantidad de instancias por categoría

Walking       48394
Jogging       48394
Upstairs      48394
Downstairs    48394
Sitting       48394
Standing      48394
Name: Actividad, dtype: int64

Volver a visualizar las cinco primeras líneas del DataFrame para revisar que todo esté en orden *(head)*


In [23]:
data_balanceado.head() # Comprobamos que las primeras lineas del DataFrame estén en orden

,Actividad,x,y,z
597,Walking,0.844462,8.008764,2.792171
598,Walking,1.116869,8.621680,3.786457
599,Walking,-0.503953,16.657684,1.307553
600,Walking,4.794363,10.760075,-1.184970
601,Walking,-0.040861,9.234595,-0.694638


Separar las variables predictoras (X) de las ctegorias o etiquetas (y)

In [25]:
X = data_balanceado[['x','y','z']] #se separan ambas variables en 2 listas para trabajar por separado
Y = data_balanceado['Actividad']

Estandarizar los datos numericos (*StandardScaler)*

*Para el caso de la estandarización de los datos se pueden usar distintas técnicas, como la normalización 01, estandarización entre -1 y 1 o el denominado Z-Score. En este caso usaremos la función Standard Scaler de Python con sus valores por defecto.*


In [27]:
X = StandardScaler().fit_transform(X)
data_estandarizada = pd.DataFrame(data = X, columns = ['x','y','z'])
data_estandarizada.head()
# Se utiliza la función Standar Scaler para estandarizar los datos de las mediciones

,x,y,z
0,-0.034488,0.076981,0.266479
1,0.017286,0.187561,0.495743
2,-0.290770,1.637381,-0.075846
3,0.716235,0.573360,-0.650576
4,-0.202754,0.298140,-0.537515


Como último paso debe reconstruir el dataframe pegando las columnas de los variables predicadores (X) y las categorías (y)


In [29]:
data_estandarizada['Actividad'] = Y.values 
data_estandarizada.head()
#finalmente se agrega los valores pertenecientes a las categorias 'Actividad' correspondiente

,x,y,z,Actividad
0,-0.034488,0.076981,0.266479,Walking
1,0.017286,0.187561,0.495743,Walking
2,-0.290770,1.637381,-0.075846,Walking
3,0.716235,0.573360,-0.650576,Walking
4,-0.202754,0.298140,-0.537515,Walking
